In [1]:
import torch
import torch.nn as nn
import numpy as np
import pandas as pd
from lib.data import ImageTokenDataset, ImageTokenDatasetAbstractCategories
from lib.models import (
    ConditionalTransformerDecoderConfig, ConditionalTransformerDecoder,
    VanillaTransformerDecoderConfig, VanillaTransformerDecoder
)
from lib import filepaths

In [2]:
config = ConditionalTransformerDecoderConfig(
    d_model=128,
    n_layers=6,
    n_heads=4,
    class_prompt_length=5,
    n_classes=10
)

model = ConditionalTransformerDecoder(config)

#params: 1459200


In [3]:
model(torch.randint(0,1024,(6,256)), torch.arange(6)).logits.shape

torch.Size([6, 256, 1024])

In [8]:
config = VanillaTransformerDecoderConfig(
    d_model=128,
    n_layers=6,
    n_heads=4
)

model = VanillaTransformerDecoder(config)

#params: 1452928


In [12]:
model(torch.randint(0,1024,(6,256))).logits.shape

torch.Size([6, 256, 1024])